In [182]:
import requests
import json
import pandas as pd
import numpy as np

graph_id = "QmYkEQD9MFq2tGcB7QX3Znn8H2FNrridwjd9CcKaFVNqJW"
url = f"https://api.thegraph.com/subgraphs/id/{graph_id}"


PASS = "PASS"
FAIL = "FAIL"
MATCH_PERCENT_TOLERANCE = 0.00001 # Much be within 0.001% of eachother
PAGE_LENGTH = 1000

check_summary = {}


In [240]:
# Helpers

def list_items_match(a: list, b: list) -> list:
    exact_match = (a == b)
    within_tolerance = abs(1 - a / b) < MATCH_PERCENT_TOLERANCE
    return exact_match | within_tolerance 
    
def perform_request(query, variables = {}):
    res = requests.post(url, json={"query": query, "variables": variables})
    return json.loads(res.text)["data"]

def perform_pagenation_request(pageable_query, key, varibales={}):
    all_found = False
    skip = 0
    data = []

    while(not all_found):
        query_variables = varibales
        query_variables["page_length"] = PAGE_LENGTH
        query_variables["skip"] = skip

        res = requests.post(url, json={"query": pageable_query, "variables": query_variables}) 

        new_data = json.loads(res.text)["data"][key]
        data += new_data

        all_found = (len(new_data) != PAGE_LENGTH)
        skip += PAGE_LENGTH
    
    return {key: data}




In [241]:
# Check total accounts equals protocol account
accounts_query = """query($skip: Int!, $page_length: Int!) {
    accounts(first: $page_length, skip: $skip) {
        id
    }
}"""

protocol_query = """query {
    protocols {
        cumulativeUniqueUsers
    }
}"""

account_data = perform_pagenation_request(accounts_query, "accounts")
protocol_data = perform_request(protocol_query)
num_accounts = len(account_data["accounts"])
cumulative_users = protocol_data["protocols"][0]["cumulativeUniqueUsers"]

result = PASS if num_accounts == cumulative_users else FAIL 

print(f"num_accounts: {num_accounts}, cumulative_users: {cumulative_users}, result: {result}")

check_summary["unique_user_check"] = result


num_accounts: 460, cumulative_users: 460, result: PASS


In [242]:
# Check cumulative for protocol match sum for pools

query = """query {
    lendingProtocols {
        totalValueLockedUSD
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeTotalRevenueUSD
        totalDepositBalanceUSD
        cumulativeDepositUSD
        totalBorrowBalanceUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
    markets {
        totalValueLockedUSD
        _cumulativeSupplySideRevenueUSD
        _cumulativeProtocolSideRevenueUSD
        _cumulativeTotalRevenueUSD
        totalDepositBalanceUSD
        cumulativeDepositUSD
        totalBorrowBalanceUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""

data = perform_request(query)
protcol_data = data["lendingProtocols"][0]
market_data = data["markets"]
df = pd.DataFrame(market_data)
df.rename(columns = {'_cumulativeSupplySideRevenueUSD':'cumulativeSupplySideRevenueUSD', '_cumulativeProtocolSideRevenueUSD':'cumulativeProtocolSideRevenueUSD', "_cumulativeTotalRevenueUSD": "cumulativeTotalRevenueUSD"}, inplace = True)
df = df.apply(pd.to_numeric, errors='coerce') 
market_sums = df.sum(axis=0)

compare_df = pd.DataFrame({"market_sums": market_sums, "protocol_cumulatives": protcol_data})
compare_df = compare_df.apply(pd.to_numeric, errors='coerce') 
compare_df["result"] = np.where(list_items_match(compare_df["market_sums"], compare_df["protocol_cumulatives"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL

check_summary["protocol_cumulatives_match_check"] = result



                                   market_sums  protocol_cumulatives result
totalValueLockedUSD               3.397133e+08          3.397133e+08   PASS
cumulativeSupplySideRevenueUSD    5.447125e+06          5.447125e+06   PASS
cumulativeProtocolSideRevenueUSD  3.406500e+07          3.406500e+07   PASS
cumulativeTotalRevenueUSD         3.951212e+07          3.951212e+07   PASS
totalDepositBalanceUSD            3.310131e+08          3.310131e+08   PASS
cumulativeDepositUSD              3.395354e+08          3.395354e+08   PASS
totalBorrowBalanceUSD             3.066000e+08          3.066000e+08   PASS
cumulativeBorrowUSD               3.298500e+08          3.298500e+08   PASS
cumulativeLiquidateUSD            0.000000e+00          0.000000e+00   PASS


In [243]:
# Check financial snapshots add up to protocol_cumulatives

snapshot_query = """query($skip: Int!, $page_length: Int!) {
    financialsDailySnapshots(first: $page_length, skip: $skip) {
        dailySupplySideRevenueUSD
        dailyProtocolSideRevenueUSD
        dailyDepositUSD
        dailyBorrowUSD
        dailyLiquidateUSD
    }
}"""

protocol_query = """query {
    lendingProtocols {
        cumulativeSupplySideRevenueUSD
        cumulativeProtocolSideRevenueUSD
        cumulativeDepositUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""


protocol_data = perform_request(protocol_query)["lendingProtocols"][0]
snapshot_data = perform_pagenation_request(snapshot_query, "financialsDailySnapshots")["financialsDailySnapshots"]
df = pd.DataFrame(snapshot_data)
df = df.apply(pd.to_numeric, errors='coerce') 
df.rename(columns = {'dailySupplySideRevenueUSD':'cumulativeSupplySideRevenueUSD', 'dailyProtocolSideRevenueUSD':'cumulativeProtocolSideRevenueUSD', "dailyDepositUSD": "cumulativeDepositUSD", "dailyBorrowUSD": "cumulativeBorrowUSD", "dailyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
sums = df.sum(axis=0)

compare_df = pd.DataFrame({"snapshot_sums": sums, "protocol_cumulatives": protocol_data})
compare_df = compare_df.apply(pd.to_numeric, errors='coerce') 
compare_df["result"] = np.where(list_items_match(compare_df["snapshot_sums"], compare_df["protocol_cumulatives"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL

check_summary["financial_snapshots_check"] = result


                                  snapshot_sums  protocol_cumulatives result
cumulativeSupplySideRevenueUSD     2.063669e+08          5.447125e+06   FAIL
cumulativeProtocolSideRevenueUSD   3.406500e+07          3.406500e+07   PASS
cumulativeDepositUSD               3.395354e+08          3.395354e+08   PASS
cumulativeBorrowUSD                3.298500e+08          3.298500e+08   PASS
cumulativeLiquidateUSD             0.000000e+00          0.000000e+00   PASS


In [244]:
# Check market snapshots add up to market cumulatives

market_query = """query {
    markets(first: 1) {
        id
        cumulativeDepositUSD
        cumulativeBorrowUSD
        cumulativeLiquidateUSD
    }
}"""

hourly_snapshot_query = """query($skip: Int!, $page_length: Int!, $market: String!) {
    marketHourlySnapshots(first: $page_length, skip: $skip, where: {market: $market}) {
        hourlyDepositUSD
        hourlyBorrowUSD
        hourlyLiquidateUSD
    }
}"""

daily_snapshot_query = """query($skip: Int!, $page_length: Int!, $market: String!) {
    marketDailySnapshots(first: $page_length, skip: $skip, where: {market: $market}) {
        dailyDepositUSD
        dailyBorrowUSD
        dailyLiquidateUSD
    }
}"""

data = perform_request(query) 
market_data = perform_request(market_query)["markets"][0]
market_id = market_data["id"]
filtered_market_data = {"cumulativeDepositUSD": float(market_data["cumulativeDepositUSD"]), "cumulativeBorrowUSD": float(market_data["cumulativeBorrowUSD"]), "cumulativeLiquidateUSD": float(market_data["cumulativeLiquidateUSD"])}

query_variables = {"market": market_id}
hourly_snapshot_data = perform_pagenation_request(hourly_snapshot_query, "marketHourlySnapshots", query_variables)["marketHourlySnapshots"]
hourly_snapshot_df = pd.DataFrame(hourly_snapshot_data)
hourly_snapshot_df = hourly_snapshot_df.apply(pd.to_numeric, errors='coerce') 
hourly_snapshot_df.rename(columns = {'hourlyDepositUSD':'cumulativeDepositUSD', 'hourlyBorrowUSD':'cumulativeBorrowUSD', "hourlyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
hourly_snapshot_sums = hourly_snapshot_df.sum(axis=0) 


daily_snapshot_data = perform_pagenation_request(daily_snapshot_query, "marketDailySnapshots", query_variables)["marketDailySnapshots"]
daily_snapshot_df = pd.DataFrame(daily_snapshot_data)
daily_snapshot_df = daily_snapshot_df.apply(pd.to_numeric, errors='coerce') 
daily_snapshot_df.rename(columns = {'dailyDepositUSD':'cumulativeDepositUSD', 'dailyBorrowUSD':'cumulativeBorrowUSD', "dailyLiquidateUSD": "cumulativeLiquidateUSD"}, inplace = True)
daily_snapshot_sums = daily_snapshot_df.sum(axis=0) 

compare_df = pd.DataFrame({"market_data": filtered_market_data, "hourly_snapshot_sums": hourly_snapshot_sums, "daily_snapshot_sums": daily_snapshot_sums})
compare_df["result"] = np.where(list_items_match(compare_df["market_data"], compare_df["hourly_snapshot_sums"]) & list_items_match(compare_df["market_data"], compare_df["daily_snapshot_sums"]), PASS, FAIL)
print(compare_df)

result = PASS if (compare_df["result"].to_numpy() == PASS).all() else FAIL
check_summary["market_snapshots_check"] = result



                        market_data  hourly_snapshot_sums  \
cumulativeDepositUSD     21250090.0              21250090   
cumulativeBorrowUSD      21250000.0              21250000   
cumulativeLiquidateUSD          0.0                     0   

                        daily_snapshot_sums result  
cumulativeDepositUSD               21250090   PASS  
cumulativeBorrowUSD                21250000   PASS  
cumulativeLiquidateUSD                    0   PASS  


In [245]:
# Summarize results

print(check_summary)

{'unique_user_check': 'PASS', 'protocol_cumulatives_match_check': 'PASS', 'financial_snapshots_check': 'FAIL', 'market_snapshots_check': 'PASS'}
